In [1]:
import datetime
from pyspark import SparkConf, SparkContext, HiveContext
sc.stop()
conf = SparkConf().set("spark.executor.instances", 32).set("spark.driver.maxResultSize", "32g")
sc = SparkContext(conf=conf)
#sc.setCheckpointDir('/user/kposminin/checkpointdir/')

In [101]:
 sc.stop()

In [2]:

def repart(filename):
    starttime = datetime.datetime.now()
    sc.textFile(filename).repartition(32*8).saveAsTextFile('.'.join(filename.split('.')[:-1]))
    print('End. Time of work {0}.'.format(datetime.datetime.now() - starttime))
#repart("/user/kposminin/la_20160818.txt")

In [13]:
from pyspark.mllib.tree import GradientBoostedTrees, GradientBoostedTreesModel
from pyspark.mllib.util import MLUtils
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.evaluation import BinaryClassificationMetrics
import sklearn
import sklearn.ensemble

# Load and parse the data file.
train = sc.textFile("/user/kposminin/la_20160817") \
  .filter(lambda s: (s[0] == '1') or (hash(s) % 500 == 0)) \
  .map(lambda r:r.split('\t')) \
  .map(lambda r:[int(e) for e in r]) \
  .collect()
test = sc.textFile("/user/kposminin/la_20160824") \
  .filter(lambda s: (s[0] == '1') or (hash(s) % 500 == 0)) \
  .map(lambda r:r.split('\t')) \
  .map(lambda r:[int(e) for e in r]) \
  .collect()

test_rdd = sc.textFile("/user/kposminin/la_20160824") \
  .map(lambda r:r.split('\t')) \
  .map(lambda r:[int(e) for e in r])


### Варьируем размер семплирования  

In [100]:
import numpy as np

modelGBT = {}
AUCROC={}

for f in [1,2,5,10,20]:
    train1 = [r for r in train if (r[0] == 1) or int(np.random.rand()*f) == 0]
    test1 = [r for r in test if (r[0] == 1) or int(np.random.rand()*f) == 0]
    s = len(train1)
    print(s)
    modelGBT[s] = sklearn.ensemble.GradientBoostingClassifier(n_estimators=500, learning_rate=0.1,
       max_depth=6, random_state=0).fit(X = [e[1:] for e in train1], y = [e[0] for e in train1])
    AUCROC[s] = sklearn.metrics.roc_auc_score(
        y_true = [e[0] for e in test1], 
        y_score = [r[1] for r in modelGBT[s].predict_proba([e[1:] for e in test1])]
    )
    AUCROC['train '+str(s)] = sklearn.metrics.roc_auc_score(
        y_true = [e[0] for e in train1], 
        y_score = [r[1] for r in modelGBT[s].predict_proba([e[1:] for e in train1])]
    )
    AUCROC['smax'+str(s)] = sklearn.metrics.roc_auc_score(
        y_true = [e[0] for e in test1], 
        y_score = [e[1] for e in test1]
    )
    AUCROC['train smax'+str(s)] = sklearn.metrics.roc_auc_score(
        y_true = [e[0] for e in train1], 
        y_score = [e[1] for e in train1]
    )

34624
18098


ValueError: Found arrays with inconsistent numbers of samples: [18098 34624]

In [63]:
print('AUCROC  score from sample number:\n' + '\n'.join(['{0} {1:.5f}'.format(k,v) for (k,v) in sorted(AUCROC.items())]))

AUCROC  score from sample number:
34624 0.94222
smax34624 0.87267
train 34624 0.99972
train smax34624 0.84838


In [4]:
def write_table_to_file(table, filename):
    f = open(filename,'w+')
    #f.write('label,' + ','.join(columns)+'\n')
    f.write('\n'.join([','.join([str(e) for e in r]) for r in table]))
    f.close()

In [14]:
columns = '''smax ,savg ,ssum ,smedian ,sstd ,cntrepeat ,cntuniq 
,duration ,mobile ,emailru ,vkru ,okru ,social_other , s1 ,s2 ,s3 ,s4 ,s5 ,s6 ,s7 ,s8 ,s9 ,s10 , 
sm1 ,sm2 ,sm3 ,sm4 ,sm5'''.replace(' ','').replace('\n','').split(',')

### Варьируем количество деревьев 

In [46]:
# Train a GradientBoostedTrees model.
import sklearn.ensemble
import sklearn
modelGBT = {}
AUCROC={}
for n in [30,50,80,100,200,300,500,800,1200]:
    modelGBT[n] = sklearn.ensemble.GradientBoostingClassifier(n_estimators=n, learning_rate=0.1,
       max_depth=6, random_state=0).fit(X = [e[1:] for e in train], y = [e[0] for e in train])
    AUCROC[n] = sklearn.metrics.roc_auc_score(
        y_true = [e[0] for e in test], 
        y_score = [r[1] for r in modelGBT[n].predict_proba([e[1:] for e in test])]
    )
    AUCROC['train '+str(n)] = sklearn.metrics.roc_auc_score(
        y_true = [e[0] for e in train], 
        y_score = [r[1] for r in modelGBT[n].predict_proba([e[1:] for e in train])]
    )
AUCROC['smax'] = sklearn.metrics.roc_auc_score(
        y_true = [e[0] for e in test], 
        y_score = [e[1] for e in test]
)
AUCROC['train smax'] = sklearn.metrics.roc_auc_score(
        y_true = [e[0] for e in train], 
        y_score = [e[1] for e in train]
)
#print(AUCROC)
# Evaluate model on test instances and compute test error
##predictions = model.predict(test.map(lambda x: x.features))
#predictionAndLabels = test.map(lambda lp: (float(model.predict(lp.features)), lp.label))

In [47]:
print('AUCROC test sample score from GBT trees number:\n' + '\n'.join(['{0} {1:.5f}'.format(k,v) for (k,v) in sorted(AUCROC.items())]))

AUCROC test sample score from GBT trees number:
30 0.93127
50 0.93696
80 0.94222
100 0.94382
200 0.94284
300 0.94261
500 0.93597
800 0.90724
1200 0.90574
smax 0.87267
train 100 0.99993
train 1200 1.00000
train 200 0.99999
train 30 0.99650
train 300 1.00000
train 50 0.99787
train 500 1.00000
train 80 0.99972
train 800 1.00000
train smax 0.84838


#### Если брать более 100 деревьев, то происходит переобучение на выборке.

#### Вариация глубины дерева

In [52]:
{'smax':AUCROC['smax'], 'train smax':AUCROC['train smax']}

{'smax': 0.87267016262338948, 'train smax': 0.84838146256114544}

In [54]:
modelGBT = {}
AUCROC = {'smax':AUCROC['smax'], 'train smax':AUCROC['train smax']}
for m in [1,3,5,7,9,11,15]:
    modelGBT[m] = sklearn.ensemble.GradientBoostingClassifier(n_estimators=100, learning_rate=0.1,
       max_depth=m, random_state=0).fit(X = [e[1:] for e in train], y = [e[0] for e in train])
    AUCROC[m] = sklearn.metrics.roc_auc_score(
        y_true = [e[0] for e in test], 
        y_score = [r[1] for r in modelGBT[m].predict_proba([e[1:] for e in test])]
    )
    AUCROC['train '+str(m)] = sklearn.metrics.roc_auc_score(
        y_true = [e[0] for e in train], 
        y_score = [r[1] for r in modelGBT[m].predict_proba([e[1:] for e in train])]
    )


In [56]:
print('AUCROC test sample score from GBT tree depth:\n' + '\n'.join(['{0} {1:.5f}'.format(k,v) for (k,v) in sorted(AUCROC.items())]))

AUCROC test sample score from GBT tree depth:
1 0.93120
3 0.92649
5 0.92779
7 0.93013
9 0.91935
11 0.85654
15 0.90053
smax 0.87267
train 1 0.98808
train 11 1.00000
train 15 1.00000
train 3 0.99584
train 5 0.99929
train 7 0.99998
train 9 1.00000
train smax 0.84838


#### До глубины 7 разница несущественна, далее начинается переобучение.

In [10]:
import sklearn
res = zip([e[0] for e in test], [e[1] for e in modelGBT.predict_proba([e[1:] for e in test])])
AUCROC = sklearn.metrics.roc_auc_score(y_true = [e[0] for e in res], y_score = [e[1] for e in res])
print('Test AUCROC {0}'.format(AUCROC))

Test AUCROC 0.913370268124


In [ ]:
res1 = test_rdd.map(lambda r:(float(r[0]),modelGBT.predict_proba(r[1:])[0][1])).collect()
ar = sklearn.metrics.roc_auc_score(y_true = [e[0] for e in res1], y_score = [e[1] for e in res1])

In [ ]:
open('test.txt','w').write(str(AUCROC)+','+str(ar)+'\n'+str(sorted(zip(columns,modelGBT.feature_importances_), key = lambda r:-r[1])))

In [ ]:
print('Test Error = ' + str(testErr))
print('Learned classification GBT model:')
print(model.toDebugString())
print('AUCROC: {0}'.format(sklearn.metrics.roc_auc_score(
            labelsAndPredictions.map(lambda r:r[0]).collect(),
            labelsAndPredictions.map(lambda r:r[1]).collect()
)))


In [ ]:
print('AUCROC smax: {0}'.format(sklearn.metrics.roc_auc_score(
            test.map(lambda lp: lp.label).collect(),
            test.map(lambda lp: lp.features[0]).collect()
)))

In [59]:
print('\n'.join(['{0} {1:.5f}'.format(k,v) for (k,v) in sorted(zip(columns,modelGBT[5].feature_importances_), key = lambda r:-r[1])]))

smax 0.13844
s1 0.11298
sm1 0.10010
s2 0.09305
ssum 0.07244
savg 0.06488
duration 0.06053
sstd 0.04256
s3 0.04210
s4 0.03787
s5 0.03752
smedian 0.02722
cntrepeat 0.02636
s10 0.02193
sm5 0.02015
cntuniq 0.01583
s6 0.01453
s8 0.01242
sm4 0.01025
sm2 0.00985
s7 0.00788
vkru 0.00674
sm3 0.00650
s9 0.00633
emailru 0.00529
mobile 0.00464
okru 0.00156
social_other 0.00002
